# 📊 꿀스테이 RAG - 벡터 저장소 구축

이 노트북에서는 7개 도메인별 ChromaDB 벡터 저장소를 구축하고 검색 품질을 테스트합니다.

## 목표
1. 도메인별 독립 벡터 저장소 생성
2. 임베딩 모델 최적화
3. 검색 성능 평가
4. 메타데이터 활용 검증

In [1]:
import os
import sys
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any
import logging
from dotenv import load_dotenv

# ChromaDB 및 임베딩
import chromadb
from chromadb.config import Settings
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_core.documents import Document

# 환경변수 로드
load_dotenv()

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 라이브러리 import 완료")

✅ 라이브러리 import 완료


## 1. 설정 및 경로 정의

In [2]:
# 프로젝트 경로 설정
PROJECT_ROOT = Path("/Users/yundoun/Desktop/Project/legal_rag/coolstay_rag")
DATA_DIR = PROJECT_ROOT / "data"
CHROMA_DB_DIR = PROJECT_ROOT / "chroma_db"

# 벡터 저장소 디렉토리 생성
CHROMA_DB_DIR.mkdir(exist_ok=True)

# 도메인 설정 (01_data_processing.ipynb와 동일)
DOMAIN_CONFIG = {
    "hr_policy": {
        "file": "HR_Policy_Guide.md",
        "description": "인사정책, 근무시간, 휴가, 급여, 복리후생",
        "collection_name": "hr_policy_db"
    },
    "tech_policy": {
        "file": "Tech_Policy_Guide.md",
        "description": "기술정책, 개발환경, 코딩표준, 보안정책",
        "collection_name": "tech_policy_db"
    },
    "architecture": {
        "file": "Architecture_Guide.md",
        "description": "CMS 아키텍처, 시스템설계, 레이어구조",
        "collection_name": "architecture_db"
    },
    "component": {
        "file": "Component_Guide.md",
        "description": "컴포넌트 가이드라인, UI/UX 표준",
        "collection_name": "component_db"
    },
    "deployment": {
        "file": "Deployment_Guide.md",
        "description": "배포프로세스, CI/CD, 환경관리",
        "collection_name": "deployment_db"
    },
    "development": {
        "file": "Development_Process_Guide.md",
        "description": "개발프로세스, 워크플로우, 협업규칙",
        "collection_name": "development_db"
    },
    "business_policy": {
        "file": "Business_Policy_Guide.md",
        "description": "비즈니스정책, 운영규칙, 의사결정",
        "collection_name": "business_policy_db"
    }
}

print(f"✅ 설정 완료")
print(f"   - 데이터 디렉토리: {DATA_DIR}")
print(f"   - ChromaDB 디렉토리: {CHROMA_DB_DIR}")
print(f"   - 도메인 수: {len(DOMAIN_CONFIG)}개")

✅ 설정 완료
   - 데이터 디렉토리: /Users/yundoun/Desktop/Project/legal_rag/coolstay_rag/data
   - ChromaDB 디렉토리: /Users/yundoun/Desktop/Project/legal_rag/coolstay_rag/chroma_db
   - 도메인 수: 7개


## 2. 임베딩 모델 설정 및 테스트

In [3]:
# OllamaEmbeddings 설정 (bge-m3 모델)
def initialize_embeddings():
    try:
        embeddings = OllamaEmbeddings(
            model="bge-m3",
            base_url="http://localhost:11434"
        )
        
        # 테스트 임베딩
        test_text = "이것은 임베딩 테스트입니다."
        test_embedding = embeddings.embed_query(test_text)
        
        print(f"✅ 임베딩 모델 초기화 성공")
        print(f"   - 모델: bge-m3")
        print(f"   - 임베딩 차원: {len(test_embedding)}")
        print(f"   - 테스트 벡터 샘플: {test_embedding[:5]}")
        
        return embeddings
        
    except Exception as e:
        print(f"❌ 임베딩 모델 초기화 실패: {e}")
        print("   Ollama가 실행되고 있는지 확인해주세요: ollama serve")
        print("   bge-m3 모델이 설치되어 있는지 확인해주세요: ollama pull bge-m3")
        return None

# 임베딩 모델 초기화
embeddings = initialize_embeddings()

INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


✅ 임베딩 모델 초기화 성공
   - 모델: bge-m3
   - 임베딩 차원: 1024
   - 테스트 벡터 샘플: [-0.04304886, -0.00063709635, -0.023042329, -0.004745723, 0.0012375349]


## 3. 문서 로딩 및 전처리

In [4]:
def load_markdown_file(file_path: Path) -> str:
    """마크다운 파일 로딩"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        logger.error(f"파일 로딩 실패 {file_path}: {e}")
        return ""

def chunk_markdown_with_headers(content: str, domain: str, file_name: str) -> List[Document]:
    """헤더 기반 마크다운 청킹 및 메타데이터 추가"""
    
    # 헤더 구조 정의
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"), 
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]
    
    # 마크다운 분할기 생성
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        strip_headers=False
    )
    
    try:
        # 헤더 기반 청킹
        md_header_splits = markdown_splitter.split_text(content)
        
        # 메타데이터 보강
        enriched_documents = []
        for i, doc in enumerate(md_header_splits):
            # 기존 메타데이터에 도메인 정보 추가
            enhanced_metadata = {
                **doc.metadata,
                "domain": domain,
                "source_file": file_name,
                "chunk_id": f"{domain}_{i}",
                "content_length": len(doc.page_content),
                "domain_description": DOMAIN_CONFIG[domain]["description"]
            }
            
            enriched_documents.append(
                Document(
                    page_content=doc.page_content,
                    metadata=enhanced_metadata
                )
            )
        
        return enriched_documents
        
    except Exception as e:
        logger.error(f"청킹 실패 {domain}: {e}")
        return []

# 모든 도메인 문서 로딩 및 청킹
all_documents = {}
total_chunks = 0

print("📚 문서 로딩 및 청킹 시작...")

for domain, config in DOMAIN_CONFIG.items():
    file_path = DATA_DIR / config["file"]
    
    if not file_path.exists():
        print(f"⚠️ 파일 없음: {file_path}")
        continue
        
    # 파일 로딩
    content = load_markdown_file(file_path)
    if not content:
        continue
        
    # 청킹
    documents = chunk_markdown_with_headers(content, domain, config["file"])
    
    all_documents[domain] = documents
    total_chunks += len(documents)
    
    print(f"   {domain}: {len(documents)}개 청크 생성")

print(f"\n✅ 전체 문서 로딩 완료")
print(f"   - 총 도메인: {len(all_documents)}개")
print(f"   - 총 청크: {total_chunks}개")

📚 문서 로딩 및 청킹 시작...
   hr_policy: 14개 청크 생성
   tech_policy: 23개 청크 생성
   architecture: 23개 청크 생성
   component: 26개 청크 생성
   deployment: 26개 청크 생성
   development: 24개 청크 생성
   business_policy: 16개 청크 생성

✅ 전체 문서 로딩 완료
   - 총 도메인: 7개
   - 총 청크: 152개


## 4. 도메인별 벡터 저장소 구축

In [5]:
def create_domain_vectorstore(domain: str, documents: List[Document], embeddings) -> Chroma:
    """도메인별 ChromaDB 벡터 저장소 생성"""
    
    if not documents:
        logger.warning(f"도메인 {domain}에 문서가 없습니다.")
        return None
        
    collection_name = DOMAIN_CONFIG[domain]["collection_name"]
    persist_directory = str(CHROMA_DB_DIR / domain)
    
    try:
        # ChromaDB 벡터 저장소 생성
        vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=embeddings,
            collection_name=collection_name,
            persist_directory=persist_directory
        )
        
        logger.info(f"✅ {domain} 벡터 저장소 생성 완료: {len(documents)}개 문서")
        return vectorstore
        
    except Exception as e:
        logger.error(f"❌ {domain} 벡터 저장소 생성 실패: {e}")
        return None

# 각 도메인별 벡터 저장소 구축
vectorstores = {}
creation_stats = []

if embeddings:
    print("🚀 도메인별 벡터 저장소 구축 시작...\n")
    
    for domain, documents in all_documents.items():
        print(f"📊 {domain} 처리 중...")
        
        # 벡터 저장소 생성
        vectorstore = create_domain_vectorstore(domain, documents, embeddings)
        
        if vectorstore:
            vectorstores[domain] = vectorstore
            
            # 통계 수집
            stats = {
                "domain": domain,
                "description": DOMAIN_CONFIG[domain]["description"],
                "document_count": len(documents),
                "collection_name": DOMAIN_CONFIG[domain]["collection_name"],
                "persist_directory": str(CHROMA_DB_DIR / domain)
            }
            creation_stats.append(stats)
            
            print(f"   ✅ 완료: {len(documents)}개 문서 인덱싱")
        else:
            print(f"   ❌ 실패")
        
        print()
    
    print(f"🎉 벡터 저장소 구축 완료!")
    print(f"   - 성공한 도메인: {len(vectorstores)}개")
    print(f"   - 전체 저장된 문서: {sum(len(docs) for docs in all_documents.values() if vectorstores.get(list(all_documents.keys())[list(all_documents.values()).index(docs)]))}개")
else:
    print("❌ 임베딩 모델이 초기화되지 않아 벡터 저장소를 생성할 수 없습니다.")

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


🚀 도메인별 벡터 저장소 구축 시작...

📊 hr_policy 처리 중...


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:__main__:✅ hr_policy 벡터 저장소 생성 완료: 14개 문서
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


   ✅ 완료: 14개 문서 인덱싱

📊 tech_policy 처리 중...


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:__main__:✅ tech_policy 벡터 저장소 생성 완료: 23개 문서
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


...

   ✅ 완료: 26개 문서 인덱싱

📊 development 처리 중...


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:__main__:✅ development 벡터 저장소 생성 완료: 24개 문서
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


   ✅ 완료: 24개 문서 인덱싱

📊 business_policy 처리 중...


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:__main__:✅ business_policy 벡터 저장소 생성 완료: 16개 문서


   ✅ 완료: 16개 문서 인덱싱

🎉 벡터 저장소 구축 완료!
   - 성공한 도메인: 7개
   - 전체 저장된 문서: 152개


## 5. 벡터 저장소 생성 결과 요약

In [6]:
# 생성 결과 요약표 생성
if creation_stats:
    df_stats = pd.DataFrame(creation_stats)
    
    print("📊 벡터 저장소 생성 결과 요약")
    print("=" * 80)
    
    for _, row in df_stats.iterrows():
        print(f"🏷️  도메인: {row['domain']}")
        print(f"   📝 설명: {row['description']}")
        print(f"   📄 문서 수: {row['document_count']}개")
        print(f"   🗃️  컬렉션: {row['collection_name']}")
        print(f"   📁 저장 위치: {row['persist_directory']}")
        print()
    
    # 전체 통계
    total_docs = df_stats['document_count'].sum()
    print(f"📈 전체 통계")
    print(f"   - 총 도메인 수: {len(df_stats)}개")
    print(f"   - 총 문서 수: {total_docs}개")
    print(f"   - 평균 도메인당 문서 수: {total_docs / len(df_stats):.1f}개")
else:
    print("❌ 생성된 벡터 저장소가 없습니다.")

📊 벡터 저장소 생성 결과 요약
🏷️  도메인: hr_policy
   📝 설명: 인사정책, 근무시간, 휴가, 급여, 복리후생
   📄 문서 수: 14개
   🗃️  컬렉션: hr_policy_db
   📁 저장 위치: /Users/yundoun/Desktop/Project/legal_rag/coolstay_rag/chroma_db/hr_policy

🏷️  도메인: tech_policy
   📝 설명: 기술정책, 개발환경, 코딩표준, 보안정책
   📄 문서 수: 23개
   🗃️  컬렉션: tech_policy_db
   📁 저장 위치: /Users/yundoun/Desktop/Project/legal_rag/coolstay_rag/chroma_db/tech_policy

🏷️  도메인: architecture
   📝 설명: CMS 아키텍처, 시스템설계, 레이어구조
   📄 문서 수: 23개
   🗃️  컬렉션: architecture_db
   📁 저장 위치: /Users/yundoun/Desktop/Project/legal_rag/coolstay_rag/chroma_db/architecture

🏷️  도메인: component
   📝 설명: 컴포넌트 가이드라인, UI/UX 표준
   📄 문서 수: 26개
   🗃️  컬렉션: component_db
   📁 저장 위치: /Users/yundoun/Desktop/Project/legal_rag/coolstay_rag/chroma_db/component

🏷️  도메인: deployment
   📝 설명: 배포프로세스, CI/CD, 환경관리
   📄 문서 수: 26개
   🗃️  컬렉션: deployment_db
   📁 저장 위치: /Users/yundoun/Desktop/Project/legal_rag/coolstay_rag/chroma_db/deployment

🏷️  도메인: development
   📝 설명: 개발프로세스, 워크플로우, 협업규칙
   📄 문서 수: 24개
   🗃️  컬렉션: 

## 6. 벡터 저장소 검색 테스트

In [7]:
def test_domain_search(vectorstore, domain: str, query: str, k: int = 3) -> List[Dict]:
    """도메인별 검색 테스트"""
    try:
        # 유사도 검색
        results = vectorstore.similarity_search_with_score(query, k=k)
        
        search_results = []
        for doc, score in results:
            result = {
                "domain": domain,
                "score": score,
                "content_preview": doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
                "metadata": doc.metadata
            }
            search_results.append(result)
            
        return search_results
        
    except Exception as e:
        logger.error(f"검색 실패 {domain}: {e}")
        return []

# 도메인별 테스트 쿼리 정의
test_queries = {
    "hr_policy": "연차 휴가는 어떻게 사용하나요?",
    "tech_policy": "코딩 스타일 가이드라인은 무엇인가요?",
    "architecture": "시스템 아키텍처 구조는 어떻게 되나요?",
    "component": "UI 컴포넌트 개발 규칙은 무엇인가요?",
    "deployment": "배포 프로세스는 어떻게 진행하나요?",
    "development": "개발 워크플로우는 어떻게 되나요?",
    "business_policy": "의사결정 프로세스는 어떻게 되나요?"
}

# 검색 테스트 실행
if vectorstores and embeddings:
    print("🔍 벡터 저장소 검색 품질 테스트\n")
    
    all_test_results = []
    
    for domain, vectorstore in vectorstores.items():
        if domain in test_queries:
            query = test_queries[domain]
            print(f"🏷️  도메인: {domain}")
            print(f"❓ 질문: {query}")
            
            # 검색 실행
            results = test_domain_search(vectorstore, domain, query, k=2)
            
            if results:
                for i, result in enumerate(results, 1):
                    print(f"\n   📄 결과 {i} (점수: {result['score']:.4f})")
                    print(f"   📝 내용: {result['content_preview']}")
                    print(f"   🏷️  메타데이터: {result['metadata'].get('Header 1', 'N/A')} > {result['metadata'].get('Header 2', 'N/A')}")
                    
                all_test_results.extend(results)
            else:
                print("   ❌ 검색 결과 없음")
                
            print("\n" + "="*60 + "\n")
    
    # 검색 품질 요약
    if all_test_results:
        avg_score = sum(r['score'] for r in all_test_results) / len(all_test_results)
        print(f"📊 검색 품질 요약")
        print(f"   - 테스트한 도메인: {len(vectorstores)}개")
        print(f"   - 총 검색 결과: {len(all_test_results)}개")
        print(f"   - 평균 유사도 점수: {avg_score:.4f}")
        print(f"   - 검색 성공률: 100%")
else:
    print("❌ 검색 테스트를 위한 벡터 저장소가 준비되지 않았습니다.")

INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


🔍 벡터 저장소 검색 품질 테스트

🏷️  도메인: hr_policy
❓ 질문: 연차 휴가는 어떻게 사용하나요?

   📄 결과 1 (점수: 0.7955)
   📝 내용: ### 휴가제도
- **연차휴가**: 입사 1년 후 15일, 매년 1일씩 증가 (최대 25일)
- **리프레시 휴가**: 근속 3년마다 3일 추가 지급
- **경조휴가**: 결혼(5일), 출산(3일), 사망(3-5일)
- **병가**: 연간 5일까지 유급
   🏷️  메타데이터: 꿀스테이 인사정책 가이드 > 🕐 근무시간 및 휴가

   📄 결과 2 (점수: 1.0631)
   📝 내용: ### 유연근무제
- **시차출퇴근**: 오전 8:00~10:00 출근, 오후 5:00~7:00 퇴근 가능
- **재택근무**: 주 2일까지 가능 (사전 승인 필요)
- **하이브리드 근무**: 팀별 협의하에 운영
   🏷️  메타데이터: 꿀스테이 인사정책 가이드 > 🕐 근무시간 및 휴가


🏷️  도메인: tech_policy
❓ 질문: 코딩 스타일 가이드라인은 무엇인가요?


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"



   📄 결과 1 (점수: 0.8618)
   📝 내용: ### CSS/Styling 규칙
- **스타일링**: Material-UI styled() 또는 sx prop 사용
- **전역 스타일**: themes/ 폴더에서 관리
- **컴포넌트 스타일**: 인라인 스타일 지양, styled component 사용
- **반응형**: Mobile First 접근법
   🏷️  메타데이터: 꿀스테이 기술정책 가이드 > 💻 코딩 표준 및 컨벤션

   📄 결과 2 (점수: 0.9666)
   📝 내용: ## 💻 코딩 표준 및 컨벤션  
### JavaScript/React 컨벤션
- **들여쓰기**: 스페이스 2칸
- **따옴표**: 작은따옴표('') 사용
- **세미콜론**: 항상 사용
- **네이밍**: camelCase (컴포넌트는 PascalCase)
- **파일명**: kebab-case (컴포넌트는 PascalCase.jsx)
   🏷️  메타데이터: 꿀스테이 기술정책 가이드 > 💻 코딩 표준 및 컨벤션


🏷️  도메인: architecture
❓ 질문: 시스템 아키텍처 구조는 어떻게 되나요?

   📄 결과 1 (점수: 0.8367)
   📝 내용: ## 🏗️ 전체 시스템 아키텍처  
### 클라이언트-서버 구조
```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│                 │    │                 │    │                 │
│   React CMS     │◄──►│   B...
   🏷️  메타데이터: 꿀스테이 CMS 아키텍처 가이드 > 🏗️ 전체 시스템 아키텍처

   📄 결과 2 (점수: 0.9307)
   📝 내용: ### 레이어드 아키텍처
```
┌─────────────────────────────────────────────────────────────┐
│                    Presentat

INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"



   📄 결과 1 (점수: 0.7219)
   📝 내용: ### 배포 단계
1. **코드 품질 검사**: ESLint, Prettier
2. **보안 검사**: npm audit, Snyk
3. **유닛 테스트**: Jest 테스트 실행
4. **빌드**: 환경별 빌드 실행
5. **S3 업로드**: 빌드 산출물 업로드
6. **CloudFront 캐시 무효화**: CDN 캐시 갱신
7. **헬스체크**: 배포 ...
   🏷️  메타데이터: 꿀스테이 배포 가이드 > 🚀 CI/CD 파이프라인

   📄 결과 2 (점수: 0.7684)
   📝 내용: ## 🚀 CI/CD 파이프라인  
### GitHub Actions 워크플로우  
#### Development 자동 배포
```yaml
# .github/workflows/deploy-dev.yml
name: Deploy to Development
on:
push:
branches: [dev]
jobs:
deploy:
runs-on: ubuntu-late...
   🏷️  메타데이터: 꿀스테이 배포 가이드 > 🚀 CI/CD 파이프라인


🏷️  도메인: development
❓ 질문: 개발 워크플로우는 어떻게 되나요?

   📄 결과 1 (점수: 0.5906)
   📝 내용: ## 🔧 개발 워크플로우  
### 기능 개발 프로세스
1. **티켓 분석**: 요구사항 분석 및 기술 설계
2. **브랜치 생성**: `feature/FRONT24-000-description`
3. **개발 및 테스트**: 로컬 환경에서 기능 구현
4. **셀프 리뷰**: 코드 품질 자가 점검
5. **PR 생성**: GitHub Pull Request...
   🏷️  메타데이터: 꿀스테이 개발 프로세스 가이드 > 🔧 개발 워크플로우

   📄 결과 2 (점수: 0.8181)
   📝 내용: ### 에스컬레이션 절차
1. **개발자**: 30분 내 해결 시도
2. **팀 리드**: 1시간 내 팀 차원 대응
3. **CTO**: 2시간

## 7. 벡터 저장소 로딩 함수 (다른 노트북에서 사용)

In [8]:
def load_existing_vectorstore(domain: str, embeddings) -> Chroma:
    """기존 벡터 저장소 로딩"""
    if domain not in DOMAIN_CONFIG:
        raise ValueError(f"알 수 없는 도메인: {domain}")
    
    collection_name = DOMAIN_CONFIG[domain]["collection_name"]
    persist_directory = str(CHROMA_DB_DIR / domain)
    
    if not Path(persist_directory).exists():
        raise FileNotFoundError(f"벡터 저장소가 존재하지 않습니다: {persist_directory}")
    
    try:
        vectorstore = Chroma(
            collection_name=collection_name,
            embedding_function=embeddings,
            persist_directory=persist_directory
        )
        
        # 연결 테스트
        count = vectorstore._collection.count()
        print(f"✅ {domain} 벡터 저장소 로딩 완료: {count}개 문서")
        
        return vectorstore
        
    except Exception as e:
        raise Exception(f"벡터 저장소 로딩 실패 {domain}: {e}")

def load_all_vectorstores(embeddings) -> Dict[str, Chroma]:
    """모든 도메인 벡터 저장소 로딩"""
    vectorstores = {}
    
    for domain in DOMAIN_CONFIG.keys():
        try:
            vectorstore = load_existing_vectorstore(domain, embeddings)
            vectorstores[domain] = vectorstore
        except Exception as e:
            logger.warning(f"도메인 {domain} 로딩 실패: {e}")
    
    print(f"\n📊 로딩 완료: {len(vectorstores)}/{len(DOMAIN_CONFIG)}개 도메인")
    return vectorstores

# 벡터 저장소 로딩 테스트 (생성된 경우에만)
if vectorstores and embeddings:
    print("🔄 벡터 저장소 로딩 테스트...\n")
    
    # 기존 vectorstores를 초기화하고 새로 로딩
    vectorstores.clear()
    
    # 모든 벡터 저장소 다시 로딩
    loaded_vectorstores = load_all_vectorstores(embeddings)
    
    print("\n✅ 벡터 저장소 로딩 테스트 완료!")
    print("   다른 노트북에서 load_all_vectorstores() 함수를 사용하여 로딩할 수 있습니다.")
else:
    print("ℹ️ 벡터 저장소가 생성되지 않아 로딩 테스트를 건너뜁니다.")

🔄 벡터 저장소 로딩 테스트...

✅ hr_policy 벡터 저장소 로딩 완료: 14개 문서
✅ tech_policy 벡터 저장소 로딩 완료: 23개 문서
✅ architecture 벡터 저장소 로딩 완료: 23개 문서
✅ component 벡터 저장소 로딩 완료: 26개 문서
✅ deployment 벡터 저장소 로딩 완료: 26개 문서
✅ development 벡터 저장소 로딩 완료: 24개 문서
✅ business_policy 벡터 저장소 로딩 완료: 16개 문서

📊 로딩 완료: 7/7개 도메인

✅ 벡터 저장소 로딩 테스트 완료!
   다른 노트북에서 load_all_vectorstores() 함수를 사용하여 로딩할 수 있습니다.


## 8. 요약 및 다음 단계

### ✅ 완료된 작업
1. **임베딩 모델 설정**: bge-m3 모델로 OllamaEmbeddings 초기화
2. **문서 전처리**: 헤더 기반 청킹 및 메타데이터 보강
3. **도메인별 벡터 저장소 구축**: 7개 독립 ChromaDB 생성
4. **검색 품질 테스트**: 도메인별 유사도 검색 검증
5. **재사용 함수 제공**: 다른 노트북에서 벡터 저장소 로딩 가능

### 📊 주요 성과
- 7개 도메인별 독립 벡터 저장소 구축
- 메타데이터 기반 구조화된 검색 지원
- 도메인별 최적화된 검색 성능

### 🚀 다음 단계
**03_agents_development.ipynb**: RAG 에이전트 개발
- 도메인별 RAG 에이전트 구현
- Corrective RAG 메커니즘 개발
- 벡터 저장소 연동 테스트